In [12]:
import time
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.datasets import fetch_20newsgroups
import torch

# Automatically download missing NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Custom print functions
def print_timer_info(message):
    print(f"[TIMER INFO] {message}")

def print_output_data(message):
    print(f"[OUTPUT DATA] {message}")

# Check CUDA availability and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print_timer_info("Checking CUDA availability...")
if device.type == 'cuda':
    num_devices = torch.cuda.device_count()
    print_output_data(f"CUDA is available. Number of GPUs: {num_devices}")
    for i in range(num_devices):
        print_output_data(f"Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print_output_data("CUDA is not available. Using CPU.")
print_timer_info(f"CUDA check completed. Using device: {device}")

# Load the dataset
start_time = time.time()
print_timer_info("Loading the 20 Newsgroups dataset...")
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
df = pd.DataFrame({'Text': newsgroups.data, 'Category': newsgroups.target})
df['Category Name'] = df['Category'].apply(lambda x: newsgroups.target_names[x])
print_timer_info(f"Time taken to load dataset: {time.time() - start_time:.2f} seconds")

# Pre-processing
print_timer_info("Starting pre-processing...")
preprocess_start_time = time.time()

df['Text'] = df['Text'].str.lower()  # Lowercasing
df['Text'] = df['Text'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))  # Remove punctuation/special characters
df['Tokens'] = df['Text'].apply(word_tokenize)  # Tokenization

stop_words = set(stopwords.words('english'))
df['Tokens'] = df['Tokens'].apply(lambda tokens: [word for word in tokens if word not in stop_words])  # Stopwords removal

stemmer = PorterStemmer()
df['Tokens'] = df['Tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])  # Stemming

print_timer_info(f"Total time for pre-processing: {time.time() - preprocess_start_time:.2f} seconds")

# Move processed data to GPU if available (Example - converting text data to tensor)
max_token_length = max(df['Tokens'].apply(len))
print_output_data(f"Maximum length of tokenized text: {max_token_length}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shalo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shalo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[TIMER INFO] Checking CUDA availability...
[OUTPUT DATA] CUDA is available. Number of GPUs: 1
[OUTPUT DATA] Device 0: NVIDIA GeForce RTX 2060
[TIMER INFO] CUDA check completed. Using device: cuda
[TIMER INFO] Loading the 20 Newsgroups dataset...
[TIMER INFO] Time taken to load dataset: 1.72 seconds
[TIMER INFO] Starting pre-processing...
[TIMER INFO] Total time for pre-processing: 28.34 seconds
[OUTPUT DATA] Maximum length of tokenized text: 6620
